This program generates a data set containing two 4D spheres that are separated along one dimension. The grand tour algorithm is applied to the data and the SVM score for each frame in the grand tour is calculated. 

In [32]:
import numpy as np
import pandas as pd
import time
import sklearn



#VARIABLES
stepSize = 0.01
nSteps = 1000
nPoints = 500
data = np.zeros((4,nPoints))
b = 2*np.pi*np.random.uniform(0,1,4)
classification = np.zeros(nPoints)
colour = np.chararray((len(classification)),unicode = True)
for i in range(0,nPoints):
    a = 2*np.pi*np.random.uniform(0,1,4)
    r = np.random.rand()
    if i%2 == 0:
        data[0,i] = r*np.cos(a[0]) + 1
        classification[i] = 0
    elif i%2 == 1:
        data[0,i] = np.cos(a[0]) - 1
        classification[i] = 1
    data[1,i] = r*np.sin(a[0])*np.cos(a[1])
    data[2,i] = r*np.sin(a[0])*np.sin(a[1])*np.cos(a[2])
    data[3,i] = r*np.sin(a[0])*np.sin(a[1])*np.sin(a[2])
    
for i in range(nPoints):
    if classification[i] == 1:
        colour[i] = 'r'
    elif classification[i] == 0:
        colour[i] = 'b'

d = np.shape(data)[0]       




def getAlpha(d):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.
    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d
    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane
    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane
    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    Alpha = getAlpha(d)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData, tBeta


tData, tBeta = GrandTour(data, nSteps)



In [33]:
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
  
SVMaccuracy = []

for i in range(nSteps):
    xData = tData[i][0]
    yData = tData[i][1]
    nData = np.vstack((xData, yData)).T
    
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)

    score = clf.score(nData, classification)
    SVMaccuracy.append(score)
    print(i, end = "\r")
 

In [34]:
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore


if __name__ == '__main__':
    
    plotSVM = 1
    plotNN = 0
    plotDT = 0
    plotBasis = 1
    plotTour = 1
    read = 0
    
    xAxisIndex = 1
    yAxisIndex = 0
    d = np.shape(tData)[1]
    nSteps = np.shape(tData)[0]
    app = 0
    app = QtGui.QApplication([])
    win = pg.GraphicsWindow(title = "GrandTour with SVM")
    win.setGeometry(0,0,1400,1200)
    win.setWindowTitle('GrandTour with SVM')
    pg.setConfigOptions(antialias=True) #dunno what that means

    
    def pause(): 
        global p
        if p == 0:
            p = 1
            pBtn.setText("Play")
            print(j)
        elif p == 1:
            p = 0
            pBtn.setText("Pause")
    
    def forward():
        global j
        j += 1
    def back():
        global j
        j -= 1
    
    j = 0
    p = 0
    jmax = nSteps + 1
    
    proxy = QtGui.QGraphicsProxyWidget()
    pBtn = QtGui.QPushButton("Pause")
    pBtn.clicked.connect(pause)
    proxy.setWidget(pBtn)  
    win.addItem(proxy,row=3,col=1)
    
    fproxy = QtGui.QGraphicsProxyWidget()
    fBtn = QtGui.QPushButton("Forward")
    fBtn.clicked.connect(forward)
    fproxy.setWidget(fBtn)  
    win.addItem(fproxy,row=3,col=2)
    
    bproxy = QtGui.QGraphicsProxyWidget()
    bBtn = QtGui.QPushButton("Back")
    bBtn.clicked.connect(back)
    bproxy.setWidget(bBtn)  
    win.addItem(bproxy,row=3,col=0)
    
    if plotTour == 1:
        w1 = win.addPlot(title="Transformed Points", col = 0, colspan = 2, row = 0, rowspan = 2)
        w1.setGeometry(0,100,14,12)
        nPoints = np.shape(tData)[2]
        s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
        pos = np.zeros((nPoints,2))
        pos[:,0] = tData[0,xAxisIndex,:]
        pos[:,1] = tData[0,yAxisIndex,:]
        if read == 1:
            s1 = pg.ScatterPlotItem(pos=pos, brush = colour, size=0.03, pxMode=False,pen=pg.mkPen(None))
        else: s1 = pg.ScatterPlotItem(pos=pos, size=0.03, pxMode=False,pen=pg.mkPen(None))
        w1.addItem(s1)
        w1.setLabel(text = "Transformed Basis: " + str(xAxisIndex), axis = 'bottom' )
        w1.setLabel(text = "Transformed Basis: " + str(yAxisIndex), axis = 'left' )
        w1.setXRange(-1.5,1.5,padding = None)
        w1.setYRange(-1.5,1.5,padding = None)
    
    if plotSVM == 1 or plotNN == 1 or plotDT == 1:
        jmax = np.argmax(accuracy)
        w2 = win.addPlot(title="Accuracy Plots", col = 0, colspan = 2,row = 2)
        w2.setXRange(0,100,padding = None)
        w2.setYRange(0,1,padding = None)
        SVMcurve = w2.plot(pen='r')
        showSVM = pg.TextItem(text = 'SVM: ' + str(accuracy[0]))
        showSVM.setParentItem(w2)
        NNcurve = w2.plot(pen='b')
        showNN = pg.TextItem(text = 'NN: ' + str(accuracy[0]))
        #showNN.setParentItem(w2)
        DTcurve = w2.plot(pen='g')
        showDT = pg.TextItem(text = 'DT: ' + str(accuracy[0]))
        #showDT.setParentItem(w2)
        jmax = np.argmax(accuracy)
    
    if plotBasis == 1:
        basis = np.arange(0,d)
        w3 = win.addPlot(title= "x Basis", row = 1, col = 2)
        w3.setYRange(0,1,padding = None)
        w4 = win.addPlot(title= "y Basis", row = 2, col = 2)
        w4.setYRange(0,1,padding = None)
    
    
    
    
    def update():
        global j, p
        
        if j > np.shape(tData)[0]-1:
            j =0
            
        if plotTour == 1:    
            pos[:,0] = tData[j,xAxisIndex,:]
            pos[:,1] = tData[j,yAxisIndex,:]
            s1.setData(pos = pos,brush = colour)
        if plotSVM == 1:
            SVMcurve.setData(SVMaccuracy[0:j])
            showSVM.setText('SVM: ' + str(round(SVMaccuracy[j],3)))
        if plotNN == 1:
            NNcurve.setData(NNaccuracy[0:j])
            showSVM.setText('NN: ' + str(round(NNaccuracy[j],3)))
        if plotDT == 1:
            DTcurve.setData(DTaccuracy[0:j])
            showSVM.setText('DT: ' + str(round(DTaccuracy[j],3)))
        
            
        if plotBasis == 1:
            w3.clear()
            w4.clear()
            basis1 = pg.BarGraphItem(x = basis, height = abs(tBeta[j,xAxisIndex,:]), width=0.1, brush='g')
            w3.addItem(basis1)
            basis2 = pg.BarGraphItem(x = basis, height = abs(tBeta[j,yAxisIndex,:]), width=0.1, brush='g')
            w4.addItem(basis2)
    
       
        
            
        if not p:
            if plotSVM == 1 or plotNN == 1 or plotDT == 1:
                w2.setXRange(0,np.shape(tData)[0],padding = None)
            j += 1
    t = QtCore.QTimer()
    t.timeout.connect(update)
    t.start(5)

## 2) Spots are transform-invariant, but not identical (top-right plot). 
## In this case, drawing is almsot as fast as 1), but there is more startup 
## overhead and memory usage since each spot generates its own pre-rendered 
## image.
    sys.exit(app.exec_())
    



SystemExit: 0

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
